<h1><center>Big Data Algorithms Techniques & Platforms</center></h1>
<h2>
<hr style=" border:none; height:3px;">
<center>Assignment 4 - MapReduce and Spark</center>
<hr style=" border:none; height:3px;">
</h2>

# Introduction


<p align="justify">
<font size="3">
In this exercise you is asked to use Spark for implementing an algorithm that applies computations on documents and dataframes.
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Execute the following cell in order to initialize Spark**</font>
<hr style=" border:none; height:2px;">
</p>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar zxvf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"


import findspark
findspark.init()

#import of the SparkSession
from pyspark.sql import SparkSession

#inizialization of the Spark Session
spark = SparkSession \
    .builder \
    .appName("Assignment4") \
    .getOrCreate()

import pandas as pd

# Analysing documents


<p align="justify">
<font size="3">
We have already seen that MapReduce procedures are good in analyzing text-files.
    
The provided data comes from a scraping operation on the website https://www.vagalume.com.br/ and is available on kaggle:
    
https://www.kaggle.com/neisse
    

    
The assignment is divided in 2 parts:
    
* Part 1 is focused on MapReduce 
    
* Part 2  is focuses on dataframes
    </font>
    </p>
    
<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>Notice that  dataset is noisy and shows all the typical issues related with data coming from this procedure (duplicated entries, etc).</font>
<hr style=" border:none; height:2px;">
</p>

# Part 1 -  MapReduce
<p align="justify">
<font size="3">
In the provided folder you can find a set of documents/files containing  descriptions of songs (lyrics and additional informations). Specifically in each file:

- the first line is the idiom/language
- the second line is the title of a song
- the third line is the relative url of the song of the original website
- from fourth line on the text you find the lyrics of the song.
    </font>
    </p>

## Exercise 1 - (2 points) - Song's lyrics 

<p align="justify">
<font size="3">
Provide a Spark MapReduce procedure that reads the documents and checks how many song's lyrics appear at least two times.

In the data-interpretation of this exercise you can consider that two files represent the same lyric if the url (3rd line of each file) is the same.

 </font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>Notice that  you can reuse any code that was made available for the previous labs/assignments or that you already developed in these contexts.</font>
<hr style=" border:none; height:2px;">
</p>

**<font  size="5" color='#91053d'>The dataset is big, some cells use sample set to calculate. But I'm confident on the code, especially Ex3.1 and Ex3.2. Please read the code in detail professor, merci!</font>**




In [4]:
import os
from pyspark import SparkContext

In [1]:
# link to file address
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!unzip -uq '/content/gdrive/MyDrive/assignment_4_summary/lyrics_files_idioms.zip' -d '/content/gdrive/MyDrive/assignment_4_summary'

In [ ]:
### Write here your code

# prepare spark environment
sc = SparkContext.getOrCreate()

# read songs data
file_path = r'/content/gdrive/MyDrive/assignment_4_summary/lyrics_files_idioms'
file_list = os.listdir(file_path)

# transfer each song to a list and integrate songs
web_list = []
lines_list = []
for file in file_list:
  new_path = '/'.join([file_path,file])
  f = open(new_path)
  lines = f.readlines()
  web_list.append(lines[1])
  lines_list.append(lines)

# make rdd and map songs
songs_rdd = sc.parallelize(lines_list)
songs_map_rdd = songs_rdd.map(lambda x: (x[1],1))

# count and filter to get frequent words (>=2)
songs_reduce = songs_map_rdd.reduceByKey(lambda x,y: x+y)
songs_2_times = songs_reduce.filter(lambda x: x[1] >= 2)

# count number of frequent words
songs_2_times.count()


## Exercise 2

### 2.1 - (1 point) - Distinct songs
Provide a Spark MapReduce procedure that provides how many distinct song's lyrics are present.

Also in this case consider the uri as key: two files represent the same lyric if the url is equal.

### 2.2 - (1 point) - Chaining
According to your implementation of Exercise 1, can you chain MapReduce additional MapReduce steps for solving Exercise 2.2? 

Provide the code for 2.1 and anwer for 2.2

In [ ]:
### Write here your code followed by the answer to question 2.2

# use url to ignore duplicates
web_list = []
file_path_distinct = []
lines_list = []
for file in file_list:
  new_path = '/'.join([file_path,file])
  f = open(new_path)
  lines = f.readlines()
  # ignore duplicated songs
  if lines[1] not in web_list:
    web_list.append(lines[1])
    file_path_distinct.append(new_path)
    lines_list.append(lines)


# make rdd and extract songs url
songs_rdd = sc.parallelize(lines_list)
songs_map_rdd = songs_rdd.map(lambda x: x[1])

# count distinct songs
songs_map_rdd.count()

12745

# Exercise 3

### 3.1 - (3 points) - Most common word for language

Now that you discovered the duplicated documents consider just one occurence of each song's lyric and define a MapReduce procedure that finds the most common word for each language (of course you must remove stop words).




**<font  size="5" color='#91053d'>The dataset is big, some cells use sample set to calculate. But I'm confident on the code, especially Ex3.1 and Ex3.2. Please read the code in detail professor, merci!</font>**

In [90]:
### Write here your code

# def func to split text to words
def text_process(text):
  import string
  import re
  import nltk 
  # remove punct
  def remove_punct(text):
      text  = "".join([char for char in text if char not in string.punctuation])
      text = re.sub('[0-9]+', '', text)
      return text
  # token
  def tokenization(text):
      text = re.split('\W+', text)
      return text
  # remove stop word
  nltk.download('stopwords')
  stopword = nltk.corpus.stopwords.words('english')
  def remove_stopwords(text):
      text = [word for word in text if word not in stopword]
      return text

  return remove_stopwords(tokenization(remove_punct(text)))

# prepare def for count word in a song
def count_words(word_list):
  word_num_list = []
  for word in word_list:
    word_num = (word, word_list.count(word))
    word_num_list.append(word_num)
  return word_num_list

# prepare def for count word of songs by language
def word_num_gp(word_num_list):
  # dict to sum word_num for words
  word_num_gp_dict = {}
  for word_num in word_num_list:
    if word_num[0] in word_num_gp_dict.keys():
      word_num_gp_dict[word_num[0]] += word_num[1]
    else:
      word_num_gp_dict[word_num[0]] = word_num[1]
  # transfer dict to listd
  word_num_gp = []
  for word in word_num_gp_dict:
    word_num = word_num_gp_dict[word]
    word_num_tuple = (word, word_num)
    word_num_gp.append(word_num_tuple)
  # return gp list
  return word_num_gp

# prepare def for rank word_num tuple for a language
def top_word_num(word_num_list):
  sorted_l=sorted(word_num_list, key=lambda t:t[1], reverse=True)
  top_word_num = sorted_l[0]
  return top_word_num

# songs words list by language (for each song)
songs_words_by_lang = songs_rdd.map(lambda x: (''.join(str.split(x[0],'\n')), count_words(text_process(x[3]))))

# words_num tuple by language (for songs in one language)
word_num_by_lang = songs_words_by_lang.reduceByKey(lambda x,y: x+y).map(lambda x: (x[0], word_num_gp(x[1])))

# sort most frequent word by language
top_word_by_lang = word_num_by_lang.map(lambda x: (x[0], top_word_num(x[1])))
top_word_by_lang.collect()

[('ENGLISH', ('I', 1430195)),
 ('SPANISH', ('que', 34703)),
 ('NA', ('oh', 21806)),
 ('KINYARWANDA', ('ni', 470)),
 ('GERMAN', ('Du', 622)),
 ('FRENCH', ('de', 598)),
 ('FINNISH', ('jotain', 64)),
 ('SWEDISH', ('ah', 1296)),
 ('ESTONIAN', ('welcome', 225)),
 ('INDONESIAN', ('girls', 144)),
 ('CATALAN', ('Rambla', 64)),
 ('PORTUGUESE', ('que', 225021)),
 ('ITALIAN', ('che', 2486)),
 ('HAITIAN_CREOLE', ('pa', 1089)),
 ('POLISH', ('czy', 121)),
 ('NORWEGIAN', ('han', 144))]

### 3.2 - (3 points) - Most common end/start words

Finally discover, for each language, the most common ending and starting word (of course, also in this case) you must remove stop words).

In [89]:
### Write here your code

# songs start-end words list by language (for each song)
# here extract start-end word by get the first and last element of word list produced by text_process function (here use -2 to get element because -1 is '')
songs_words_by_lang = songs_rdd.map(lambda x: (''.join(str.split(x[0],'\n')), count_words([text_process(x[3])[0]]+[text_process(x[3])[-2]])))

# words_num tuple by language (for songs in one language)
word_num_by_lang = songs_words_by_lang.reduceByKey(lambda x,y: x+y).map(lambda x: (x[0], word_num_gp(x[1])))

# sort most frequent word by language
top_word_by_lang = word_num_by_lang.map(lambda x: (x[0], top_word_num(x[1])))
top_word_by_lang.collect()


[('ENGLISH', ('I', 831)),
 ('SPANISH', ('No', 14)),
 ('NA', ('Instrumental', 141)),
 ('KINYARWANDA', ('Romanji', 2)),
 ('GERMAN', ('Ich', 3)),
 ('FRENCH', ('chorus', 4)),
 ('FINNISH', ('Kerran', 1)),
 ('SWEDISH', ('Ah', 1)),
 ('ESTONIAN', ('Im', 1)),
 ('INDONESIAN', ('Ya', 1)),
 ('CATALAN', ('Rambla', 1)),
 ('PORTUGUESE', ('Eu', 416)),
 ('ITALIAN', ('Il', 2)),
 ('HAITIAN_CREOLE', ('Men', 1)),
 ('POLISH', ('nie', 1)),
 ('NORWEGIAN', ('lyrics', 1))]


<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**DataFrames**</font>
<hr style=" border:none; height:2px;">
</p>

# Part 2 - Dataframes

In this part you can use Pandas Dataframes or  Spark Dataframes.  I suggest to use a Spark Dataframe
end exploit the Pandas functionalities as we have seen in the 2nd assignment. Download the two available datasets at the link:

https://www.kaggle.com/neisse/scrapped-lyrics-from-6-genres

You can find two .cvs files: 

* artists-data.csv

* lyrics-data.csv


# Import artist data.
<p align="justify">
<font size="3">
The artist data in the .csv file can be stored in a dataframe. 
    
Each row of the .csv file describes an artist and the columns represent the following data:
    
* Artist - The artist's name
* Popularity - Popularity score at the date of scrapping
* ALink - The link to the artist's page
* AGenre - Primary musical genre of the artist
* AGenres - A list (pay attention to the format) of genres the artist fits in
    
</font>
</p>


# Import song's lyrics data.


<p align="justify">
<font size="3">
    
Each row of the .csv file describes a lyric and the columns represent the following data:
    
* ALink - The link to the webpage of the artist
* SLink - The link to the webpage of the song
* Idiom - The idiom of the lyric
* Lyric - The lyrics
* SName - The name of the song

    

</font>
</p>


#  Exercise 4 - (3 points) - Artist's genre

<p align="justify">
<font size="3">
Provide a program that finds the artists for which the genre is not specified.

</font>
</p>






In [ ]:
from pyspark.sql.functions import col,isnan,when,count

In [ ]:
### Write here your code

# link to file address
from google.colab import drive
drive.mount('/content/gdrive')

artists_path = '/content/gdrive/MyDrive/assignment_4_summary/archive/artists-data.csv'
lyrics_path = '/content/gdrive/MyDrive/assignment_4_summary/archive/lyrics-data.csv'
dfs_art = spark.read.csv(artists_path, header=True, inferSchema=True)
dfs_lrc = spark.read.csv(lyrics_path, header=True, inferSchema=True)

print(dfs_art.where(col('Genre').isNull()).collect())
print(dfs_art.filter(isnan(dfs_art.Genre)).collect())
print(dfs_art.where(col('Genre').isNull()).count())
print(dfs_art.filter(isnan(dfs_art.Genre)).count())


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
[]
[]
0
0


#  Exercise 5 - (3 points) - Duplicates

<p align="justify">
<font size="3">
Provide a program that removes the duplicates in the artists (also in this case the URL is the key).

</font>
</p>




In [ ]:
### Write here your code

# explore
print('row number with duplicate:')
print(dfs_art.count())

print('number of distinct artist:')
print(dfs_art.select('Artist').distinct().count())
print('number of distinct url:')
print(dfs_art.select('Link').distinct().count())

# remove duplicate
dfs_art_no_dplc = dfs_art.dropDuplicates(subset=[c for c in dfs_art.columns if c in ['Artist', 'Link']])

print('row number after remove duplicate')
print(dfs_art_no_dplc.count())
dfs_art_no_dplc.show(5)

row number with duplicate:
3242
number of distinct artist:
2940
number of distinct url:
2940
row number after remove duplicate
2940
+----------------+-----+----------+------------------+-----+--------------------+
|          Artist|Songs|Popularity|              Link|Genre|              Genres|
+----------------+-----+----------+------------------+-----+--------------------+
|        Bon Jovi|  359|      31.2|        /bon-jovi/| Rock|Hard Rock; Rock; ...|
|Daniel y Enrique|   12|       0.0|/daniel-y-enrique/|  Pop|    Pop; World Music|
|        Daughtry|   78|       1.4|        /daughtry/| Rock|Rock; Pop/Rock; R...|
|      ForestSide|    7|       0.0|      /forestside/| Rock|Rock Alternativo;...|
|  Mamilo Mandala|   10|       0.0|  /mamilo-mandala/| Rock|Classic Rock; Har...|
+----------------+-----+----------+------------------+-----+--------------------+
only showing top 5 rows



#  Exercise 6 - (4 points)

<p align="justify">
<font size="3">
Provide a program that using dataframe return the 100 most popular artists and the lyrics of their songs.
</font>
</p>

In [ ]:
### Write here your code
# rank
dfs_art_sorted = dfs_art_no_dplc.sort('Popularity', ascending=False)
# top artist
dfs_art_top100 = dfs_art_sorted.limit(100)
# join artist and songs
dfs_art_top100_songs = dfs_art_top100.join(dfs_lrc, dfs_art_top100.Link == dfs_lrc.ALink, "left_outer")
# extract useful columns
dfs_art_top100_songs = dfs_art_top100_songs.select('Artist','Link','Popularity','SName','SLink','Lyric')

dfs_art_top100_songs.show(5)

+-----------+--------------+----------+--------------------+--------------------+--------------------+
|     Artist|          Link|Popularity|               SName|               SLink|               Lyric|
+-----------+--------------+----------+--------------------+--------------------+--------------------+
|ExaltaSamba|/exalta-samba/|      13.0|Como Nunca Amei N...|/exalta-samba/com...|Nunca pensei que ...|
|ExaltaSamba|/exalta-samba/|      13.0|           Separação|/exalta-samba/sep...|Melhor assim.... ...|
|ExaltaSamba|/exalta-samba/|      13.0|           Telegrama|/exalta-samba/tel...|Ah, que saudade d...|
|ExaltaSamba|/exalta-samba/|      13.0|      Livre Pra Voar|/exalta-samba/liv...|"Quando a gente s...|
|ExaltaSamba|/exalta-samba/|      13.0|              É Você|/exalta-samba/e-v...|É você.. Meu dese...|
+-----------+--------------+----------+--------------------+--------------------+--------------------+
only showing top 5 rows



# 2 - Bonus 


<p align="justify">
<font size="3">
Using the approach you prefer (just Dataframes, hybrid approach)  :
    
* the 10 most common words in the lyrics of each artist
* the 10 most common words for each genre. For this question we can use the primary genre of the artist.

</font>
</p>




In [ ]:
# Write here your code and the detailed description of the MapReduce algorithm.




















